# 04 - AGGREGATION OF SR, PFF, AND COMBINE DATA BY PLAYER
- The combination of SR, PFF, and Combine data, aggregated by player
- Supposed to be a model-ready dataset at the end

In [1]:
import numpy as np
import pandas as pd
import warnings
import time

# Column and row display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.reset_option('display.max_colwidth', None)


# Notebook cell width display
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 98% !important; }</style>"))

# Float appearance, Pandas and NumPy
pd.set_option('display.float_format', '{:.0f}'.format)
np.set_printoptions(suppress=True, precision=0)

# Supress warnings
warnings.filterwarnings('ignore')

# AGGREGATION
- Starting with grouping by player, position (which should be WR), and season year
  - Future iterations of this may include tight ends and running backs, as well as player profiles in different seasons
  - Late in his career, Larry Fitzgerald lined up more in the slot and became more of a slot receiver than a true "X" receiver.

In [2]:
sportradar_wr = pd.read_csv('../working_exports/sportradar_wr.csv')

In [3]:
aggregate = sportradar_wr.groupby(['player_name', 'player_position', 'season_year'])[['reception', 'target', 'yards', 'att_yards', 'yards_after_catch', 'yards_after_contact', 
                                        'broken_tackles', 'catchable', 'touchdown', 'difficult_attempt', 'difficult_catch', 'weather_attempt', 'weather_catch', 'qb_bf_attempt',
                                        'qb_bf_catch', 'clutch_catch', 'cross_attempt', 'cross_catch', 'curl_attempt', 'curl_catch',
                                        'corner_attempt', 'corner_catch', 'out_attempt', 'out_catch',
                                        'post_attempt', 'post_catch', 'underneath_screen_attempt', 'underneath_screen_catch', 'flat_attempt', 'flat_catch', 'slant_attempt',
                                        'slant_catch', 'wr_screen_attempt', 'wr_screen_catch', 'comeback_attempt', 'comeback_catch', 'go_attempt', 'go_catch', 'in_attempt',
                                        'in_catch', 'deep_attempt', 'deep_catch', 'large_yac_catch', 'play_action_attempt', 'play_action_catch', 'rpo_attempt', 'rpo_catch',
                                        'tackle_breaker_catch', 'beast_catch', 'hurry_up_attempt', 'hurry_up_catch', 'deep_sideline_attempt', 'deep_sideline_catch', 'dpi_drawn',
                                        'possession_saver_attempt', 'possession_saver_catch', 'conversion_attempt', 'conversion_catch', 'redzone_attempt', 'redzone_catch']].sum().sort_values('yards', ascending = False).reset_index()

In [4]:
aggregate.head()

,player_name,player_position,season_year,reception,target,yards,att_yards,yards_after_catch,yards_after_contact,broken_tackles,catchable,touchdown,difficult_attempt,difficult_catch,weather_attempt,weather_catch,qb_bf_attempt,qb_bf_catch,clutch_catch,cross_attempt,cross_catch,curl_attempt,curl_catch,corner_attempt,corner_catch,out_attempt,out_catch,post_attempt,post_catch,underneath_screen_attempt,underneath_screen_catch,flat_attempt,flat_catch,slant_attempt,slant_catch,wr_screen_attempt,wr_screen_catch,comeback_attempt,comeback_catch,go_attempt,go_catch,in_attempt,in_catch,deep_attempt,deep_catch,large_yac_catch,play_action_attempt,play_action_catch,rpo_attempt,rpo_catch,tackle_breaker_catch,beast_catch,hurry_up_attempt,hurry_up_catch,deep_sideline_attempt,deep_sideline_catch,dpi_drawn,possession_saver_attempt,possession_saver_catch,conversion_attempt,conversion_catch,redzone_attempt,redzone_catch
0,Justin Jefferson,WR,2022,128,184,1809,1858,624,132,6,22,8,34,1,84,84,66,40,6,5,4,18,11,19,9,38,30,12,10,0,0,16,14,12,10,12,11,18,9,20,8,14,12,27,16,22,44,31,4,2,6,1,33,23,20,11,0,106,64,39,27,19,6
1,Tyreek Hill,WR,2022,119,170,1710,2107,482,58,5,23,7,27,0,85,85,46,33,3,16,13,18,15,7,3,22,13,19,8,0,0,7,7,18,13,14,12,17,16,23,12,9,7,38,20,17,69,50,25,15,4,0,5,5,15,10,0,111,71,34,22,7,2
2,Davante Adams,WR,2022,100,180,1516,2129,493,95,9,27,14,52,0,50,50,68,31,1,6,6,21,12,11,3,26,16,15,7,0,0,8,8,14,6,7,6,17,6,35,14,20,16,36,14,16,36,23,8,6,8,1,9,5,20,8,0,100,45,40,23,12,4
3,A.J. Brown,WR,2022,88,145,1496,1754,548,192,7,20,11,37,0,45,45,56,30,2,4,4,16,10,8,3,18,10,2,1,1,1,6,5,35,26,6,5,13,6,22,9,14,8,28,13,18,47,27,43,32,7,4,24,12,21,11,0,84,44,24,17,6,4
4,Stefon Diggs,WR,2022,108,154,1429,1729,419,100,2,18,11,27,1,80,80,58,35,4,5,3,20,17,12,5,17,12,12,6,2,2,15,13,22,18,9,9,14,8,16,9,10,6,23,12,12,46,37,26,20,2,2,16,12,14,8,0,87,55,28,17,18,7


In [5]:
aggregate.loc[0, ['cross_attempt', 'curl_attempt', 'corner_attempt', 'out_attempt', 'post_attempt', 'underneath_screen_attempt', 'flat_attempt', 'slant_attempt', 'wr_screen_attempt', 'comeback_attempt', 'go_attempt', 'in_attempt']].sum()

184

In [6]:
aggregate.loc[0, ['cross_catch', 'curl_catch', 'corner_catch', 'out_catch', 'post_catch', 'underneath_screen_catch', 'flat_catch', 'slant_catch', 'wr_screen_catch', 'comeback_catch', 'go_catch', 'in_catch']].sum()


128

## Aggregation prep

### PFF
- Data from Pro Football Focus

In [7]:
pff_wr_grades_2022 = pd.read_csv('../pff_data/csv_exports/pff_2022_wr_summary.csv')

#### New PFF columns
- Create new utilization rate columns from the PFF data

In [8]:
rates = {
        'route_rate': ('routes', 'pass_plays'),
        'slot_rate': ('slot_snaps', 'pass_plays'),
        'wide_rate': ('wide_snaps', 'pass_plays'),
        'contested_catch_rate': ('contested_receptions', 'contested_targets')
        }

for rate, (numerator, denominator) in rates.items():
    pff_wr_grades_2022[rate] = np.where(
                                        (pff_wr_grades_2022[numerator] == 0) & (pff_wr_grades_2022[denominator] == 0),
                                        0,
                                        pff_wr_grades_2022[numerator] / pff_wr_grades_2022[denominator]
                                        )

Reordering columns

In [9]:
pff_wr_grades_2022 = pff_wr_grades_2022 [['player', 'player_game_count', 'targets', 'receptions', 'contested_receptions', 'contested_targets', 'drops', 'fumbles',  
                                          'inline_snaps', 'pass_blocks', 'pass_plays', 'routes', 'slot_snaps', 'wide_snaps', 'route_rate', 'slot_rate', 'wide_rate', 'contested_catch_rate']]

In [10]:
pff_wr_grades_2022 = pff_wr_grades_2022.rename(columns={"player": "player_name"})

#### Renaming Sportradar ("SR") and PFF columns
- Different sources tend to store different spellings of names
  - Suffixes, full vs short names, middle names, commas, Roman numerals

In [11]:
# SR renaming
aggregate.loc[aggregate['player_name'] == 'Darrell Henderson Jr.', 'player_name'] = 'Darrell Henderson'
aggregate.loc[aggregate['player_name'] == 'Brian Robinson Jr.', 'player_name'] = 'Brian Robinson'
aggregate.loc[aggregate['player_name'] == 'Tyron Billy-Johnson', 'player_name'] = 'Tyron Johnson'
aggregate.loc[aggregate['player_name'] == 'Ty Montgomery II', 'player_name'] = 'Ty Montgomery'
aggregate.loc[aggregate['player_name'] == 'Deebo Samuel Sr.', 'player_name'] = 'Deebo Samuel'
aggregate.loc[aggregate['player_name'] == 'Ray-Ray McCloud', 'player_name'] = 'Ray-Ray McCloud III'
aggregate.loc[aggregate['player_name'] == 'James Proche II', 'player_name'] = 'James Proche'

In [12]:
# PFF renaming
pff_wr_grades_2022.loc[pff_wr_grades_2022['player_name'] == 'D.K. Metcalf', 'player_name'] = 'DK Metcalf'
pff_wr_grades_2022.loc[pff_wr_grades_2022['player_name'] == 'Mecole Hardman Jr.', 'player_name'] = 'Mecole Hardman'
pff_wr_grades_2022.loc[pff_wr_grades_2022['player_name'] == 'David Sills V', 'player_name'] = 'David Sills'
pff_wr_grades_2022.loc[pff_wr_grades_2022['player_name'] == 'James Proche II', 'player_name'] = 'James Proche'
pff_wr_grades_2022.loc[pff_wr_grades_2022['player_name'] == 'D\'Wayne Eskridge', 'player_name'] = 'Dee Eskridge'
pff_wr_grades_2022.loc[pff_wr_grades_2022['player_name'] == 'Tony Jones Jr.', 'player_name'] = 'Tony Jones'
pff_wr_grades_2022.loc[pff_wr_grades_2022['player_name'] == 'Demetric Felton Jr.', 'player_name'] = 'Demetric Felton'

In [13]:
aggregate = aggregate.merge(pff_wr_grades_2022, how = 'left', on = 'player_name')

Dropping the SR column for catches (`reception`) and using the PFF column (`receptions`) instead

In [14]:
aggregate = aggregate.drop(['reception'], axis=1)

Checking data types

In [15]:
aggregate.dtypes

player_name                   object
player_position               object
season_year                    int64
target                       float64
yards                        float64
att_yards                    float64
yards_after_catch            float64
yards_after_contact          float64
broken_tackles               float64
catchable                    float64
touchdown                    float64
difficult_attempt              int64
difficult_catch                int64
weather_attempt                int64
weather_catch                  int64
qb_bf_attempt                  int64
qb_bf_catch                    int64
clutch_catch                   int64
cross_attempt                  int64
cross_catch                    int64
curl_attempt                   int64
curl_catch                     int64
corner_attempt                 int64
corner_catch                   int64
out_attempt                    int64
out_catch                      int64
post_attempt                   int64
p

### Converting some columns from floats to integers

In [16]:
int_columns = ['season_year','yards','att_yards','yards_after_catch','yards_after_contact', 'broken_tackles', 'catchable', 'touchdown', 'conversion_catch', 'redzone_catch']

for col in int_columns:
    aggregate[col] = aggregate[col].fillna(0)
    aggregate[col] = aggregate[col].astype(int)

### Success rate columns
- For pass plays by route and context, I'm adding attempts and success rates (based on catches out of attempts)
- I may not end up using this features, but I'm keeping them for now

In [17]:
types = [
    "cross", "curl", "corner", "out", "post", "underneath_screen",
    "flat", "slant", "wr_screen", "comeback", "go", "in", "deep",
    "play_action", "rpo", "hurry_up", "deep_sideline", "possession_saver", "difficult"
]

for type in types:
    catch_key = f"{type}_catch"
    attempt_key = f"{type}_attempt"
    success_key = f"{type}_success_rate"
    aggregate[success_key] = np.where((aggregate[catch_key] == 0) & (aggregate[attempt_key] == 0),
                                      0, aggregate[catch_key] / aggregate[attempt_key])

### Utilization rate columns

In [18]:
types = [
        "cross", "corner", "out", "curl", "post", "underneath_screen",
        "flat", "slant", "wr_screen", "comeback", "go", "in", "deep",
        "play_action", "rpo", "hurry_up", "deep_sideline", "possession_saver", "difficult",
        "conversion", "redzone"
        ]

for type in types:
    attempt_key = f"{type}_attempt"
    pct_key = f"{type}_pct"
    aggregate[pct_key] = np.where(
                                    (aggregate[attempt_key] == 0) & (aggregate['target'] == 0),
                                    0,
                                    aggregate[attempt_key] / aggregate['target']
                                )

In [19]:
aggregate.loc[0, ['cross_pct', 'curl_pct', 'corner_pct', 'out_pct', 'post_pct', 'underneath_screen_pct', 'flat_pct', 'slant_pct', 'wr_screen_pct', 'comeback_pct', 'go_pct', 'in_pct']].sum()

0.9999999999999999

### New columns

In [20]:
# aggregate['adot'] = np.where((aggregate['att_yards'] == 0) & (aggregate['targets'] == 0), 0, aggregate['att_yards'] / aggregate['targets'])
# aggregate['avg_yac'] = np.where((aggregate['yards_after_catch'] == 0) & (aggregate['receptions'] == 0), 0, aggregate['yards_after_catch'] / aggregate['receptions'])
# aggregate['avg_yacon'] = np.where((aggregate['yards_after_contact'] == 0) & (aggregate['receptions'] == 0), 0, aggregate['yards_after_contact'] / aggregate['receptions'])
# aggregate['catch_rate'] = np.where((aggregate['receptions'] == 0) & (aggregate['targets'] == 0), 0, aggregate['receptions'] / aggregate['targets'])
# aggregate['yprr'] = np.where((aggregate['yards'] == 0) & (aggregate['routes'] == 0), 0, aggregate['yards'] / aggregate['routes'])

aggregate['adot'] = np.where((aggregate['att_yards'] == 0) & (aggregate['target'] == 0), 0, aggregate['att_yards'] / aggregate['target'])
aggregate['avg_yac'] = np.where((aggregate['yards_after_catch'] == 0) & (aggregate['receptions'] == 0), 0, aggregate['yards_after_catch'] / aggregate['receptions'])
aggregate['avg_yacon'] = np.where((aggregate['yards_after_contact'] == 0) & (aggregate['receptions'] == 0), 0, aggregate['yards_after_contact'] / aggregate['receptions'])
aggregate['catch_rate'] = np.where((aggregate['receptions'] == 0) & (aggregate['target'] == 0), 0, aggregate['receptions'] / aggregate['target'])
aggregate['yprr'] = np.where((aggregate['yards'] == 0) & (aggregate['routes'] == 0), 0, aggregate['yards'] / aggregate['routes'])

## Combine and Pro Days
- Data from the NFL Combine and player Pro Days

In [21]:
combine = pd.read_csv('../combine_data/combine.csv')

More name cleaning

In [22]:
combine.loc[combine['name'] == 'D.J. Moore', 'name'] = 'DJ Moore'
combine.loc[combine['name'] == 'Mark Ingram', 'name'] = 'Mark Ingram II'
combine.loc[combine['name'] == 'Marvin Jones', 'name'] = 'Marvin Jones Jr.'
combine.loc[combine['name'] == 'Allen Robinson', 'name'] = 'Allen Robinson II'
combine.loc[combine['name'] == 'Melvin Gordon', 'name'] = 'Melvin Gordon III'
combine.loc[combine['name'] == 'Ty Montgomery', 'name'] = 'Ty Montgomery II'
combine.loc[combine['name'] == 'D.J. Chark', 'name'] = 'DJ Chark Jr.'
combine.loc[combine['name'] == 'Ray-ray McCloud', 'name'] = 'Ray-Ray McCloud III'
combine.loc[combine['name'] == 'Trequan Smith', 'name'] = 'Tre\'Quan Smith'
combine.loc[combine['name'] == 'Jeff Wilson', 'name'] = 'Jeff Wilson Jr.'
combine.loc[combine['name'] == 'Darrell Henderson', 'name'] = 'Darrell Henderson Jr.'
combine.loc[combine['name'] == 'Irv Smith', 'name'] = 'Irv Smith Jr.'
combine.loc[combine['name'] == 'Benny Snell', 'name'] = 'Benny Snell Jr.'
combine.loc[combine['name'] == 'Gabriel Davis', 'name'] = 'Gabe Davis'
combine.loc[combine['name'] == 'Anthony McFarland', 'name'] = 'Anthony McFarland Jr.'
combine.loc[combine['name'] == 'Michael Pittman', 'name'] = 'Michael Pittman Jr.'
combine.loc[combine['name'] == 'Travis Etienne', 'name'] = 'Travis Etienne Jr.'
combine.loc[combine['name'] == 'Larry Rountree', 'name'] = 'Larry Rountree III'
combine.loc[combine['name'] == 'Velus Jones', 'name'] = 'Velus Jones Jr.'
combine.loc[combine['name'] == 'Brian Robinson', 'name'] = 'Brian Robinson Jr.'
combine.loc[combine['name'] == 'Pierre Strong', 'name'] = 'Pierre Strong Jr.'
combine.loc[combine['name'] == 'Cedarian Lamb', 'name'] = 'CeeDee Lamb'
combine.loc[combine['name'] == 'D.K. Metcalf', 'name'] = 'DK Metcalf'
combine.loc[combine['name'] == 'JaMarr Chase', 'name'] = 'Ja\'Marr Chase'
combine.loc[combine['name'] == 'Michael Pittman', 'name'] = 'Michael Pittman Jr.'
combine.loc[combine['name'] == 'D.J. Moore', 'name'] = 'DJ Moore'
combine.loc[combine['name'] == 'Gabriel Davis', 'name'] = 'Gabe Davis'
combine.loc[combine['name'] == 'Josh Palmer', 'name'] = 'Joshua Palmer'
combine.loc[combine['name'] == 'Terrace Marshall', 'name'] = 'Terrace Marshall Jr.'
combine.loc[combine['name'] == 'Bennett Skowronek', 'name'] = 'Ben Skowronek'
combine.loc[combine['name'] == 'Robby Anderson', 'name'] = 'Robbie Chosen'
combine.loc[combine['name'] == 'Laviska Shenault', 'name'] = 'Laviska Shenault Jr.'
combine.loc[combine['name'] == 'Ray-Ray Mccloud', 'name'] = 'Ray-Ray McCloud Jr.'
combine.loc[combine['name'] == 'WanDale Robinson', 'name'] = 'Wan\'Dale Robinson'
combine.loc[combine['name'] == 'K.J. Hamler', 'name'] = 'KJ Hamler'
combine.loc[combine['name'] == 'Cedrick Wilson', 'name'] = 'Cedrick Wilson Jr.'
combine.loc[combine['name'] == 'Velus Jones', 'name'] = 'Velus Jones Jr.'
combine.loc[combine['name'] == 'DWayne Eskridge', 'name'] = 'Dee Eskridge'
combine.loc[combine['name'] == 'Michael Woods', 'name'] = 'Michael Woods II'

In [23]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [24]:
combine.head()

,year,name,height_in,weight_lbs,40,bench,vertical,broad_jump,shuttle,3_cone
0,2024.00,Javon Baker,73.38,202.00,4.54,14.00,37.00,121.00,4.27,7.02
1,2024.00,Jermaine Burton,72.25,196.00,4.45,13.00,38.50,133.00,4.24,7.00
2,2024.00,Jalen Coker,73.25,208.00,4.57,14.00,42.50,128.00,4.27,7.02
3,2024.00,Keon Coleman,74.63,213.00,4.61,16.00,38.00,127.00,4.28,7.05
4,2024.00,Malachi Corley,70.63,215.00,4.55,16.00,35.50,121.00,4.28,7.05


In [25]:
combine.rename(columns={'name': 'player_name'}, inplace = True)

### Merge Combine DF with Aggregate DF

In [26]:
aggregate = aggregate.merge(combine, how = 'left', on = 'player_name')

### Checking nulls

In [27]:
aggregate.isnull().sum()

player_name                        0
player_position                    0
season_year                        0
target                             0
yards                              0
att_yards                          0
yards_after_catch                  0
yards_after_contact                0
broken_tackles                     0
catchable                          0
touchdown                          0
difficult_attempt                  0
difficult_catch                    0
weather_attempt                    0
weather_catch                      0
qb_bf_attempt                      0
qb_bf_catch                        0
clutch_catch                       0
cross_attempt                      0
cross_catch                        0
curl_attempt                       0
curl_catch                         0
corner_attempt                     0
corner_catch                       0
out_attempt                        0
out_catch                          0
post_attempt                       0
p

Won't need the year column; it's just the year they participated in the Combine or Pro Day.

In [28]:
aggregate.drop(columns = ['year'], inplace = True)

### Players not in the Combine/Pro Day scrape
- Although we imputed missing data for the players in the Combine/Pro Day scrape, that scrape is missing entries for some players
- We'll have to manually enter these values after referring to their Wikipedia pages

In [29]:
aggregate[aggregate['height_in'].isnull()][['player_name', 'height_in', 'weight_lbs', '40', 'bench', 'vertical', 'broad_jump', 'shuttle', '3_cone']]

,player_name,height_in,weight_lbs,40,bench,vertical,broad_jump,shuttle,3_cone
41,Adam Thielen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,Kalif Raymond,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Olamide Zaccheaus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,Rashid Shaheed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,Nick Westbrook-Ikhine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Kendall Hinton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,Tre'Quan Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,Ray-Ray McCloud III,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,KhaDarel Hodge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,Jamal Agnew,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
aggregate.loc[aggregate['player_name'] == 'Adam Thielen', 'height_in'] = 73.13
aggregate.loc[aggregate['player_name'] == 'Adam Thielen', 'weight_lbs'] = 192
aggregate.loc[aggregate['player_name'] == 'Adam Thielen', '40'] = 4.45
aggregate.loc[aggregate['player_name'] == 'Adam Thielen', 'vertical'] = 36
aggregate.loc[aggregate['player_name'] == 'Adam Thielen', 'broad_jump'] = 120
aggregate.loc[aggregate['player_name'] == 'Adam Thielen', 'shuttle'] = 4.49
aggregate.loc[aggregate['player_name'] == 'Adam Thielen', '3_cone'] = 6.77

In [31]:
aggregate.loc[aggregate['player_name'] == 'Olamide Zaccheaus', 'height_in'] = 68.25
aggregate.loc[aggregate['player_name'] == 'Olamide Zaccheaus', 'weight_lbs'] = 188
aggregate.loc[aggregate['player_name'] == 'Olamide Zaccheaus', '40'] = 4.49
aggregate.loc[aggregate['player_name'] == 'Olamide Zaccheaus', 'vertical'] = 35.5
aggregate.loc[aggregate['player_name'] == 'Olamide Zaccheaus', 'broad_jump'] = 125
aggregate.loc[aggregate['player_name'] == 'Olamide Zaccheaus', 'shuttle'] = 4.19
aggregate.loc[aggregate['player_name'] == 'Olamide Zaccheaus', '3_cone'] = 7
aggregate.loc[aggregate['player_name'] == 'Olamide Zaccheaus', 'bench'] = 12

In [32]:
aggregate.loc[aggregate['player_name'] == 'Rashid Shaheed', 'height_in'] = 70.63
aggregate.loc[aggregate['player_name'] == 'Rashid Shaheed', 'weight_lbs'] = 185
aggregate.loc[aggregate['player_name'] == 'Rashid Shaheed', 'bench'] = 10

In [33]:
aggregate.loc[aggregate['player_name'] == 'Nick Westbrook-Ikhine', 'height_in'] = 74.63
aggregate.loc[aggregate['player_name'] == 'Nick Westbrook-Ikhine', 'weight_lbs'] = 211

In [34]:
aggregate.loc[aggregate['player_name'] == 'Kendall Hinton', 'height_in'] = 70.38
aggregate.loc[aggregate['player_name'] == 'Kendall Hinton', 'weight_lbs'] = 193

In [35]:
aggregate.loc[aggregate['player_name'] == 'Tre\'Quan Smith', 'height_in'] = 73.75
aggregate.loc[aggregate['player_name'] == 'Tre\'Quan Smith', 'weight_lbs'] = 203
aggregate.loc[aggregate['player_name'] == 'Tre\'Quan Smith', '40'] = 4.49
aggregate.loc[aggregate['player_name'] == 'Tre\'Quan Smith', 'shuttle'] = 4.5
aggregate.loc[aggregate['player_name'] == 'Tre\'Quan Smith', '3_cone'] = 6.97
aggregate.loc[aggregate['player_name'] == 'Tre\'Quan Smith', 'vertical'] = 37.5
aggregate.loc[aggregate['player_name'] == 'Tre\'Quan Smith', 'broad_jump'] = 130
aggregate.loc[aggregate['player_name'] == 'Tre\'Quan Smith', 'bench'] = 12

In [36]:
aggregate.loc[aggregate['player_name'] == 'Ray-Ray McCloud III', 'height_in'] = 69.5
aggregate.loc[aggregate['player_name'] == 'Ray-Ray McCloud III', 'weight_lbs'] = 190
aggregate.loc[aggregate['player_name'] == 'Ray-Ray McCloud III', '40'] = 4.49
aggregate.loc[aggregate['player_name'] == 'Ray-Ray McCloud III', 'shuttle'] = 4.32
aggregate.loc[aggregate['player_name'] == 'Ray-Ray McCloud III', '3_cone'] = 6.89
aggregate.loc[aggregate['player_name'] == 'Ray-Ray McCloud III', 'vertical'] = 34.5
aggregate.loc[aggregate['player_name'] == 'Ray-Ray McCloud III', 'broad_jump'] = 113
aggregate.loc[aggregate['player_name'] == 'Ray-Ray McCloud III', 'bench'] = 13

In [37]:
aggregate.loc[aggregate['player_name'] == 'KhaDarel Hodge', 'height_in'] = 73.75
aggregate.loc[aggregate['player_name'] == 'KhaDarel Hodge', 'weight_lbs'] = 198
aggregate.loc[aggregate['player_name'] == 'KhaDarel Hodge', '40'] = 4.58
aggregate.loc[aggregate['player_name'] == 'KhaDarel Hodge', 'vertical'] = 38
aggregate.loc[aggregate['player_name'] == 'KhaDarel Hodge', 'broad_jump'] = 124

In [38]:
aggregate.loc[aggregate['player_name'] == 'Jamal Agnew', 'height_in'] = 69.5
aggregate.loc[aggregate['player_name'] == 'Jamal Agnew', 'weight_lbs'] = 186
aggregate.loc[aggregate['player_name'] == 'Jamal Agnew', '40'] = 4.34
aggregate.loc[aggregate['player_name'] == 'Jamal Agnew', 'shuttle'] = 4.27
aggregate.loc[aggregate['player_name'] == 'Jamal Agnew', '3_cone'] = 6.92
aggregate.loc[aggregate['player_name'] == 'Jamal Agnew', 'vertical'] = 36
aggregate.loc[aggregate['player_name'] == 'Jamal Agnew', 'broad_jump'] = 125
aggregate.loc[aggregate['player_name'] == 'Jamal Agnew', 'bench'] = 16

In [39]:
aggregate.loc[aggregate['player_name'] == 'Dyami Brown', 'height_in'] = 72.63
aggregate.loc[aggregate['player_name'] == 'Dyami Brown', 'weight_lbs'] = 189
aggregate.loc[aggregate['player_name'] == 'Dyami Brown', '40'] = 4.44
aggregate.loc[aggregate['player_name'] == 'Dyami Brown', 'shuttle'] = 4.35
aggregate.loc[aggregate['player_name'] == 'Dyami Brown', '3_cone'] = 6.85
aggregate.loc[aggregate['player_name'] == 'Dyami Brown', 'vertical'] = 35.5
aggregate.loc[aggregate['player_name'] == 'Dyami Brown', 'broad_jump'] = 128
aggregate.loc[aggregate['player_name'] == 'Dyami Brown', 'bench'] = 18

In [40]:
aggregate.loc[aggregate['player_name'] == 'Keelan Cole Sr.', 'height_in'] = 72.88
aggregate.loc[aggregate['player_name'] == 'Keelan Cole Sr.', 'weight_lbs'] = 194
aggregate.loc[aggregate['player_name'] == 'Keelan Cole Sr.', '40'] = 4.59
aggregate.loc[aggregate['player_name'] == 'Keelan Cole Sr.', 'shuttle'] = 4.4
aggregate.loc[aggregate['player_name'] == 'Keelan Cole Sr.', '3_cone'] = 6.69

In [41]:
aggregate.loc[aggregate['player_name'] == 'Tom Kennedy', 'height_in'] = 70.38
aggregate.loc[aggregate['player_name'] == 'Tom Kennedy', 'weight_lbs'] = 197
aggregate.loc[aggregate['player_name'] == 'Tom Kennedy', '40'] = 4.55
aggregate.loc[aggregate['player_name'] == 'Tom Kennedy', 'shuttle'] = 4.08
aggregate.loc[aggregate['player_name'] == 'Tom Kennedy', '3_cone'] = 6.66
aggregate.loc[aggregate['player_name'] == 'Tom Kennedy', 'vertical'] = 37
aggregate.loc[aggregate['player_name'] == 'Tom Kennedy', 'broad_jump'] = 124
aggregate.loc[aggregate['player_name'] == 'Tom Kennedy', 'bench'] = 22

In [42]:
aggregate.loc[aggregate['player_name'] == 'Steven Sims', 'height_in'] = 68.88
aggregate.loc[aggregate['player_name'] == 'Steven Sims', 'weight_lbs'] = 184
aggregate.loc[aggregate['player_name'] == 'Steven Sims', '40'] = 4.56
aggregate.loc[aggregate['player_name'] == 'Steven Sims', 'shuttle'] = 4.43
aggregate.loc[aggregate['player_name'] == 'Steven Sims', '3_cone'] = 7.2
aggregate.loc[aggregate['player_name'] == 'Steven Sims', 'vertical'] = 31.5
aggregate.loc[aggregate['player_name'] == 'Steven Sims', 'broad_jump'] = 115
aggregate.loc[aggregate['player_name'] == 'Steven Sims', 'bench'] = 11

In [43]:
aggregate.loc[aggregate['player_name'] == 'River Cracraft', 'height_in'] = 72.25
aggregate.loc[aggregate['player_name'] == 'River Cracraft', 'weight_lbs'] = 193
aggregate.loc[aggregate['player_name'] == 'River Cracraft', 'bench'] = 15

In [44]:
aggregate.loc[aggregate['player_name'] == 'Michael Bandy', 'height_in'] = 69.5
aggregate.loc[aggregate['player_name'] == 'Michael Bandy', 'weight_lbs'] = 195

In [45]:
aggregate.loc[aggregate['player_name'] == 'Samori Toure', 'height_in'] = 72.75
aggregate.loc[aggregate['player_name'] == 'Samori Toure', 'weight_lbs'] = 191
aggregate.loc[aggregate['player_name'] == 'Samori Toure', '40'] = 4.48
aggregate.loc[aggregate['player_name'] == 'Samori Toure', 'shuttle'] = 4.22
aggregate.loc[aggregate['player_name'] == 'Samori Toure', '3_cone'] = 6.77
aggregate.loc[aggregate['player_name'] == 'Samori Toure', 'vertical'] = 34.5
aggregate.loc[aggregate['player_name'] == 'Samori Toure', 'broad_jump'] = 124
aggregate.loc[aggregate['player_name'] == 'Samori Toure', 'bench'] = 7

In [46]:
aggregate.loc[aggregate['player_name'] == 'Jalen Virgil', 'height_in'] = 72.13
aggregate.loc[aggregate['player_name'] == 'Jalen Virgil', 'weight_lbs'] = 207
aggregate.loc[aggregate['player_name'] == 'Jalen Virgil', '40'] = 4.4
aggregate.loc[aggregate['player_name'] == 'Jalen Virgil', 'shuttle'] = 4.35
aggregate.loc[aggregate['player_name'] == 'Jalen Virgil', '3_cone'] = 7.15
aggregate.loc[aggregate['player_name'] == 'Jalen Virgil', 'vertical'] = 36.5
aggregate.loc[aggregate['player_name'] == 'Jalen Virgil', 'broad_jump'] = 130
aggregate.loc[aggregate['player_name'] == 'Jalen Virgil', 'bench'] = 19

In [47]:
aggregate.loc[aggregate['player_name'] == 'Cody Hollister', 'height_in'] = 75.25
aggregate.loc[aggregate['player_name'] == 'Cody Hollister', 'weight_lbs'] = 209
aggregate.loc[aggregate['player_name'] == 'Cody Hollister', '40'] = 4.53
aggregate.loc[aggregate['player_name'] == 'Cody Hollister', '3_cone'] = 7.02
aggregate.loc[aggregate['player_name'] == 'Cody Hollister', 'vertical'] = 36
aggregate.loc[aggregate['player_name'] == 'Cody Hollister', 'broad_jump'] = 120
aggregate.loc[aggregate['player_name'] == 'Cody Hollister', 'bench'] = 29

In [48]:
aggregate.loc[aggregate['player_name'] == 'Gunner Olszewski', 'height_in'] = 69.63
aggregate.loc[aggregate['player_name'] == 'Gunner Olszewski', 'weight_lbs'] = 178
aggregate.loc[aggregate['player_name'] == 'Gunner Olszewski', '40'] = 4.56
aggregate.loc[aggregate['player_name'] == 'Gunner Olszewski', 'shuttle'] = 4.37
aggregate.loc[aggregate['player_name'] == 'Gunner Olszewski', '3_cone'] = 6.88
aggregate.loc[aggregate['player_name'] == 'Gunner Olszewski', 'vertical'] = 36
aggregate.loc[aggregate['player_name'] == 'Gunner Olszewski', 'broad_jump'] = 118

In [49]:
aggregate.loc[aggregate['player_name'] == 'Anthony Schwartz', 'height_in'] = 72
aggregate.loc[aggregate['player_name'] == 'Anthony Schwartz', 'weight_lbs'] = 186
aggregate.loc[aggregate['player_name'] == 'Anthony Schwartz', '40'] = 4.25
aggregate.loc[aggregate['player_name'] == 'Anthony Schwartz', 'shuttle'] = 4.25
aggregate.loc[aggregate['player_name'] == 'Anthony Schwartz', '3_cone'] = 7.13
aggregate.loc[aggregate['player_name'] == 'Anthony Schwartz', 'vertical'] = 32
aggregate.loc[aggregate['player_name'] == 'Anthony Schwartz', 'broad_jump'] = 133

In [50]:
aggregate.loc[aggregate['player_name'] == 'Andre Baccellia', 'height_in'] = 68.63
aggregate.loc[aggregate['player_name'] == 'Andre Baccellia', 'weight_lbs'] = 175

In [51]:
aggregate.loc[aggregate['player_name'] == 'Brandon Johnson', 'height_in'] = 72.38
aggregate.loc[aggregate['player_name'] == 'Brandon Johnson', 'weight_lbs'] = 195
aggregate.loc[aggregate['player_name'] == 'Brandon Johnson', '40'] = 4.61
aggregate.loc[aggregate['player_name'] == 'Brandon Johnson', 'shuttle'] = 4.52
aggregate.loc[aggregate['player_name'] == 'Brandon Johnson', '3_cone'] = 7.62
aggregate.loc[aggregate['player_name'] == 'Brandon Johnson', 'vertical'] = 35
aggregate.loc[aggregate['player_name'] == 'Brandon Johnson', 'broad_jump'] = 116

In [52]:
aggregate.loc[aggregate['player_name'] == 'Dax Milne', 'height_in'] = 72.63
aggregate.loc[aggregate['player_name'] == 'Dax Milne', 'weight_lbs'] = 193
aggregate.loc[aggregate['player_name'] == 'Dax Milne', '40'] = 4.56
aggregate.loc[aggregate['player_name'] == 'Dax Milne', 'shuttle'] = 4.22
aggregate.loc[aggregate['player_name'] == 'Dax Milne', '3_cone'] = 6.85
aggregate.loc[aggregate['player_name'] == 'Dax Milne', 'vertical'] = 31
aggregate.loc[aggregate['player_name'] == 'Dax Milne', 'broad_jump'] = 114
aggregate.loc[aggregate['player_name'] == 'Dax Milne', 'bench'] = 14

In [53]:
aggregate.loc[aggregate['player_name'] == 'Cole Beasley', 'height_in'] = 67.88
aggregate.loc[aggregate['player_name'] == 'Cole Beasley', 'weight_lbs'] = 175
aggregate.loc[aggregate['player_name'] == 'Cole Beasley', '40'] = 4.49
aggregate.loc[aggregate['player_name'] == 'Cole Beasley', 'shuttle'] = 4.4
aggregate.loc[aggregate['player_name'] == 'Cole Beasley', '3_cone'] = 7.16
aggregate.loc[aggregate['player_name'] == 'Cole Beasley', 'vertical'] = 38
aggregate.loc[aggregate['player_name'] == 'Cole Beasley', 'broad_jump'] = 126
aggregate.loc[aggregate['player_name'] == 'Cole Beasley', 'bench'] = 17

In [54]:
aggregate.loc[aggregate['player_name'] == 'Deven Thompkins', 'height_in'] = 66.88
aggregate.loc[aggregate['player_name'] == 'Deven Thompkins', 'weight_lbs'] = 167
aggregate.loc[aggregate['player_name'] == 'Deven Thompkins', '40'] = 4.44
aggregate.loc[aggregate['player_name'] == 'Deven Thompkins', 'shuttle'] = 4.18
aggregate.loc[aggregate['player_name'] == 'Deven Thompkins', '3_cone'] = 6.98
aggregate.loc[aggregate['player_name'] == 'Deven Thompkins', 'vertical'] = 38.5
aggregate.loc[aggregate['player_name'] == 'Deven Thompkins', 'broad_jump'] = 132
aggregate.loc[aggregate['player_name'] == 'Deven Thompkins', 'bench'] = 9

In [55]:
aggregate.loc[aggregate['player_name'] == 'Daylen Baldwin', 'height_in'] = 73.88
aggregate.loc[aggregate['player_name'] == 'Daylen Baldwin', 'weight_lbs'] = 218
aggregate.loc[aggregate['player_name'] == 'Daylen Baldwin', '40'] = 4.71
aggregate.loc[aggregate['player_name'] == 'Daylen Baldwin', 'shuttle'] = 4.44
aggregate.loc[aggregate['player_name'] == 'Daylen Baldwin', 'vertical'] = 34.5
aggregate.loc[aggregate['player_name'] == 'Daylen Baldwin', 'broad_jump'] = 120
aggregate.loc[aggregate['player_name'] == 'Daylen Baldwin', 'bench'] = 16

In [56]:
aggregate.loc[aggregate['player_name'] == 'Dareke Young', 'height_in'] = 74
aggregate.loc[aggregate['player_name'] == 'Dareke Young', 'weight_lbs'] = 223
aggregate.loc[aggregate['player_name'] == 'Dareke Young', '40'] = 4.44
aggregate.loc[aggregate['player_name'] == 'Dareke Young', 'shuttle'] = 4.19
aggregate.loc[aggregate['player_name'] == 'Dareke Young', '3_cone'] = 6.88
aggregate.loc[aggregate['player_name'] == 'Dareke Young', 'vertical'] = 37
aggregate.loc[aggregate['player_name'] == 'Dareke Young', 'broad_jump'] = 135
aggregate.loc[aggregate['player_name'] == 'Dareke Young', 'bench'] = 22

In [57]:
aggregate.loc[aggregate['player_name'] == 'Simi Fehoko', 'height_in'] = 75.88
aggregate.loc[aggregate['player_name'] == 'Simi Fehoko', 'weight_lbs'] = 222
aggregate.loc[aggregate['player_name'] == 'Simi Fehoko', '40'] = 4.43
aggregate.loc[aggregate['player_name'] == 'Simi Fehoko', 'shuttle'] = 4.26
aggregate.loc[aggregate['player_name'] == 'Simi Fehoko', '3_cone'] = 6.78
aggregate.loc[aggregate['player_name'] == 'Simi Fehoko', 'vertical'] = 34.5
aggregate.loc[aggregate['player_name'] == 'Simi Fehoko', 'broad_jump'] = 120
aggregate.loc[aggregate['player_name'] == 'Simi Fehoko', 'bench'] = 16

In [58]:
aggregate.loc[aggregate['player_name'] == 'Dennis Houston', 'height_in'] = 73.13
aggregate.loc[aggregate['player_name'] == 'Dennis Houston', 'weight_lbs'] = 198
aggregate.loc[aggregate['player_name'] == 'Dennis Houston', '40'] = 4.49
aggregate.loc[aggregate['player_name'] == 'Dennis Houston', 'shuttle'] = 4.35
aggregate.loc[aggregate['player_name'] == 'Dennis Houston', '3_cone'] = 7.17
aggregate.loc[aggregate['player_name'] == 'Dennis Houston', 'vertical'] = 39
aggregate.loc[aggregate['player_name'] == 'Dennis Houston', 'broad_jump'] = 124
aggregate.loc[aggregate['player_name'] == 'Dennis Houston', 'bench'] = 19

In [59]:
aggregate.loc[aggregate['player_name'] == 'Ty Montgomery', 'height_in'] = 69.88
aggregate.loc[aggregate['player_name'] == 'Ty Montgomery', 'weight_lbs'] = 221
aggregate.loc[aggregate['player_name'] == 'Ty Montgomery', '40'] = 4.55
aggregate.loc[aggregate['player_name'] == 'Ty Montgomery', 'shuttle'] = 4.21
aggregate.loc[aggregate['player_name'] == 'Ty Montgomery', '3_cone'] = 6.97
aggregate.loc[aggregate['player_name'] == 'Ty Montgomery', 'vertical'] = 40.5
aggregate.loc[aggregate['player_name'] == 'Ty Montgomery', 'broad_jump'] = 121
aggregate.loc[aggregate['player_name'] == 'Ty Montgomery', 'bench'] = 16

In [60]:
aggregate.loc[aggregate['player_name'] == 'Nsimba Webster', 'height_in'] = 69
aggregate.loc[aggregate['player_name'] == 'Nsimba Webster', 'weight_lbs'] = 176
aggregate.loc[aggregate['player_name'] == 'Nsimba Webster', '40'] = 4.55
aggregate.loc[aggregate['player_name'] == 'Nsimba Webster', 'shuttle'] = 4.17
aggregate.loc[aggregate['player_name'] == 'Nsimba Webster', '3_cone'] = 6.83
aggregate.loc[aggregate['player_name'] == 'Nsimba Webster', 'vertical'] = 31.5
aggregate.loc[aggregate['player_name'] == 'Nsimba Webster', 'broad_jump'] = 121
aggregate.loc[aggregate['player_name'] == 'Nsimba Webster', 'bench'] = 11

In [61]:
aggregate.loc[aggregate['player_name'] == 'Deonte Harty', 'height_in'] = 66.38
aggregate.loc[aggregate['player_name'] == 'Deonte Harty', 'weight_lbs'] = 171
aggregate.loc[aggregate['player_name'] == 'Deonte Harty', '40'] = 4.48
aggregate.loc[aggregate['player_name'] == 'Deonte Harty', 'shuttle'] = 4.16
aggregate.loc[aggregate['player_name'] == 'Deonte Harty', '3_cone'] = 6.82
aggregate.loc[aggregate['player_name'] == 'Deonte Harty', 'vertical'] = 33
aggregate.loc[aggregate['player_name'] == 'Deonte Harty', 'broad_jump'] = 114
aggregate.loc[aggregate['player_name'] == 'Deonte Harty', 'bench'] = 10

In [62]:
aggregate.loc[aggregate['player_name'] == 'KaVontae Turpin', 'height_in'] = 68.88
aggregate.loc[aggregate['player_name'] == 'KaVontae Turpin', 'weight_lbs'] = 158
aggregate.loc[aggregate['player_name'] == 'KaVontae Turpin', '40'] = 4.31
aggregate.loc[aggregate['player_name'] == 'KaVontae Turpin', 'shuttle'] = 4.25
aggregate.loc[aggregate['player_name'] == 'KaVontae Turpin', '3_cone'] = 6.65
aggregate.loc[aggregate['player_name'] == 'KaVontae Turpin', 'vertical'] = 40
aggregate.loc[aggregate['player_name'] == 'KaVontae Turpin', 'broad_jump'] = 129
aggregate.loc[aggregate['player_name'] == 'KaVontae Turpin', 'bench'] = 2

In [63]:
aggregate.loc[aggregate['player_name'] == 'Maurice Alexander', 'height_in'] = 70.25
aggregate.loc[aggregate['player_name'] == 'Maurice Alexander', 'weight_lbs'] = 180

In [64]:
aggregate.loc[aggregate['player_name'] == 'C.J. Board', 'player_name'] = 'CJ Board'
aggregate.loc[aggregate['player_name'] == 'CJ Board', 'height_in'] = 72.63
aggregate.loc[aggregate['player_name'] == 'CJ Board', 'weight_lbs'] = 181
aggregate.loc[aggregate['player_name'] == 'CJ Board', '40'] = 4.42
aggregate.loc[aggregate['player_name'] == 'CJ Board', 'shuttle'] = 4.06
aggregate.loc[aggregate['player_name'] == 'CJ Board', '3_cone'] = 6.67
aggregate.loc[aggregate['player_name'] == 'CJ Board', 'vertical'] = 38.5
aggregate.loc[aggregate['player_name'] == 'CJ Board', 'broad_jump'] = 132
aggregate.loc[aggregate['player_name'] == 'CJ Board', 'bench'] = 9

In [65]:
aggregate.loc[aggregate['player_name'] == 'Jason Moore', 'height_in'] = 72.5
aggregate.loc[aggregate['player_name'] == 'Jason Moore', 'weight_lbs'] = 215
aggregate.loc[aggregate['player_name'] == 'Jason Moore', '40'] = 4.57
aggregate.loc[aggregate['player_name'] == 'Jason Moore', 'shuttle'] = 4.07
aggregate.loc[aggregate['player_name'] == 'Jason Moore', '3_cone'] = 6.62
aggregate.loc[aggregate['player_name'] == 'Jason Moore', 'vertical'] = 37
aggregate.loc[aggregate['player_name'] == 'Jason Moore', 'broad_jump'] = 124
aggregate.loc[aggregate['player_name'] == 'Jason Moore', 'bench'] = 17

In [66]:
aggregate.loc[aggregate['player_name'] == 'Mason Kinsey', 'height_in'] = 70.5
aggregate.loc[aggregate['player_name'] == 'Mason Kinsey', 'weight_lbs'] = 198
aggregate.loc[aggregate['player_name'] == 'Mason Kinsey', '40'] = 4.65
aggregate.loc[aggregate['player_name'] == 'Mason Kinsey', 'shuttle'] = 4.34
aggregate.loc[aggregate['player_name'] == 'Mason Kinsey', '3_cone'] = 7.3
aggregate.loc[aggregate['player_name'] == 'Mason Kinsey', 'vertical'] = 37.5
aggregate.loc[aggregate['player_name'] == 'Mason Kinsey', 'broad_jump'] = 125
aggregate.loc[aggregate['player_name'] == 'Mason Kinsey', 'bench'] = 16

In [67]:
aggregate.loc[aggregate['player_name'] == 'Montrell Washington', 'height_in'] = 68.88
aggregate.loc[aggregate['player_name'] == 'Montrell Washington', 'weight_lbs'] = 176
aggregate.loc[aggregate['player_name'] == 'Montrell Washington', '40'] = 4.48
aggregate.loc[aggregate['player_name'] == 'Montrell Washington', 'shuttle'] = 4.32
aggregate.loc[aggregate['player_name'] == 'Montrell Washington', '3_cone'] = 7.25
aggregate.loc[aggregate['player_name'] == 'Montrell Washington', 'vertical'] = 36
aggregate.loc[aggregate['player_name'] == 'Montrell Washington', 'broad_jump'] = 124
aggregate.loc[aggregate['player_name'] == 'Montrell Washington', 'bench'] = 6

In [68]:
aggregate.loc[aggregate['player_name'] == 'DJ Turner', 'height_in'] = 71.25
aggregate.loc[aggregate['player_name'] == 'DJ Turner', 'weight_lbs'] = 178
aggregate.loc[aggregate['player_name'] == 'DJ Turner', '40'] = 4.26
aggregate.loc[aggregate['player_name'] == 'DJ Turner', 'vertical'] = 38.5
aggregate.loc[aggregate['player_name'] == 'DJ Turner', 'broad_jump'] = 131

In [69]:
aggregate.loc[aggregate['player_name'] == 'Josh Ali', 'height_in'] = 71.38
aggregate.loc[aggregate['player_name'] == 'Josh Ali', 'weight_lbs'] = 191

In [70]:
aggregate.loc[aggregate['player_name'] == 'Josh Gordon', 'height_in'] = 75.13
aggregate.loc[aggregate['player_name'] == 'Josh Gordon', 'weight_lbs'] = 224
aggregate.loc[aggregate['player_name'] == 'Josh Gordon', '40'] = 4.52
aggregate.loc[aggregate['player_name'] == 'Josh Gordon', 'vertical'] = 35.5
aggregate.loc[aggregate['player_name'] == 'Josh Gordon', 'broad_jump'] = 122
aggregate.loc[aggregate['player_name'] == 'Josh Gordon', 'bench'] = 13

In [71]:
aggregate.loc[aggregate['player_name'] == 'Lance McCutcheon', 'height_in'] = 74.13
aggregate.loc[aggregate['player_name'] == 'Lance McCutcheon', 'weight_lbs'] = 207
aggregate.loc[aggregate['player_name'] == 'Lance McCutcheon', '40'] = 4.57
aggregate.loc[aggregate['player_name'] == 'Lance McCutcheon', 'shuttle'] = 4.25
aggregate.loc[aggregate['player_name'] == 'Lance McCutcheon', '3_cone'] = 6.82
aggregate.loc[aggregate['player_name'] == 'Lance McCutcheon', 'vertical'] = 36.5
aggregate.loc[aggregate['player_name'] == 'Lance McCutcheon', 'broad_jump'] = 122
aggregate.loc[aggregate['player_name'] == 'Lance McCutcheon', 'bench'] = 19

In [72]:
aggregate.loc[aggregate['player_name'] == 'Tanner Gentry', 'height_in'] = 73
aggregate.loc[aggregate['player_name'] == 'Tanner Gentry', 'weight_lbs'] = 208
aggregate.loc[aggregate['player_name'] == 'Tanner Gentry', '40'] = 4.58
aggregate.loc[aggregate['player_name'] == 'Tanner Gentry', 'shuttle'] = 4.43
aggregate.loc[aggregate['player_name'] == 'Tanner Gentry', '3_cone'] = 7.29
aggregate.loc[aggregate['player_name'] == 'Tanner Gentry', 'vertical'] = 38
aggregate.loc[aggregate['player_name'] == 'Tanner Gentry', 'broad_jump'] = 125
aggregate.loc[aggregate['player_name'] == 'Tanner Gentry', 'bench'] = 19

### Imputing missing data
- Some of the 44 players we manually entered Pro Day data for did not partake in every exercise, so we'll impute these metrics based on weight class.

In [73]:
# Find nulls
columns = aggregate.columns
field = [var for var in columns if aggregate[var].isnull().sum() != 0]
print(aggregate[field].isnull().sum())

height_in      1
weight_lbs     1
40             9
bench         14
vertical      10
broad_jump    10
shuttle       13
3_cone        13
dtype: int64


#### Height and weight

In [74]:
aggregate[aggregate['height_in'].isnull()]

,player_name,player_position,season_year,target,yards,att_yards,yards_after_catch,yards_after_contact,broken_tackles,catchable,touchdown,difficult_attempt,difficult_catch,weather_attempt,weather_catch,qb_bf_attempt,qb_bf_catch,clutch_catch,cross_attempt,cross_catch,curl_attempt,curl_catch,corner_attempt,corner_catch,out_attempt,out_catch,post_attempt,post_catch,underneath_screen_attempt,underneath_screen_catch,flat_attempt,flat_catch,slant_attempt,slant_catch,wr_screen_attempt,wr_screen_catch,comeback_attempt,comeback_catch,go_attempt,go_catch,in_attempt,in_catch,deep_attempt,deep_catch,large_yac_catch,play_action_attempt,play_action_catch,rpo_attempt,rpo_catch,tackle_breaker_catch,beast_catch,hurry_up_attempt,hurry_up_catch,deep_sideline_attempt,deep_sideline_catch,dpi_drawn,possession_saver_attempt,possession_saver_catch,conversion_attempt,conversion_catch,redzone_attempt,redzone_catch,player_game_count,targets,receptions,contested_receptions,contested_targets,drops,fumbles,inline_snaps,pass_blocks,pass_plays,routes,slot_snaps,wide_snaps,route_rate,slot_rate,wide_rate,contested_catch_rate,cross_success_rate,curl_success_rate,corner_success_rate,out_success_rate,post_success_rate,underneath_screen_success_rate,flat_success_rate,slant_success_rate,wr_screen_success_rate,comeback_success_rate,go_success_rate,in_success_rate,deep_success_rate,play_action_success_rate,rpo_success_rate,hurry_up_success_rate,deep_sideline_success_rate,possession_saver_success_rate,difficult_success_rate,cross_pct,corner_pct,out_pct,curl_pct,post_pct,underneath_screen_pct,flat_pct,slant_pct,wr_screen_pct,comeback_pct,go_pct,in_pct,deep_pct,play_action_pct,rpo_pct,hurry_up_pct,deep_sideline_pct,possession_saver_pct,difficult_pct,conversion_pct,redzone_pct,adot,avg_yac,avg_yacon,catch_rate,yprr,height_in,weight_lbs,40,bench,vertical,broad_jump,shuttle,3_cone
50,Kalif Raymond,WR,2022,64.00,616,638,218,34,2,5,0,12,0,47,47,27,17,0,3,1,12,10,3,0,8,5,7,5,2,2,4,4,6,4,8,8,5,4,4,2,2,2,13,7,9,12,9,2,2,2,0,3,3,6,2,0,27,15,10,9,2,0,17,62,47,3,7,0,1,2,2,346,322,159,181,0.93,0.46,0.52,0.43,0.33,0.83,0.00,0.62,0.71,1.00,1.00,0.67,1.00,0.80,0.50,1.00,0.54,0.75,1.00,1.00,0.33,0.56,0.00,0.05,0.05,0.12,0.19,0.11,0.03,0.06,0.09,0.12,0.08,0.06,0.03,0.20,0.19,0.03,0.05,0.09,0.42,0.19,0.16,0.03,9.97,4.64,0.72,0.73,1.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Found Raymond's height and weight on Google.

In [75]:
aggregate.loc[aggregate['player_name'] == 'Kalif Raymond', 'height_in'] = 68
aggregate.loc[aggregate['player_name'] == 'Kalif Raymond', 'weight_lbs'] = 180

#### 40-yard dash
- Using the same median-by-weight-class method we did in the Combine notebook

In [76]:
aggregate[aggregate['40'].isnull()]

,player_name,player_position,season_year,target,yards,att_yards,yards_after_catch,yards_after_contact,broken_tackles,catchable,touchdown,difficult_attempt,difficult_catch,weather_attempt,weather_catch,qb_bf_attempt,qb_bf_catch,clutch_catch,cross_attempt,cross_catch,curl_attempt,curl_catch,corner_attempt,corner_catch,out_attempt,out_catch,post_attempt,post_catch,underneath_screen_attempt,underneath_screen_catch,flat_attempt,flat_catch,slant_attempt,slant_catch,wr_screen_attempt,wr_screen_catch,comeback_attempt,comeback_catch,go_attempt,go_catch,in_attempt,in_catch,deep_attempt,deep_catch,large_yac_catch,play_action_attempt,play_action_catch,rpo_attempt,rpo_catch,tackle_breaker_catch,beast_catch,hurry_up_attempt,hurry_up_catch,deep_sideline_attempt,deep_sideline_catch,dpi_drawn,possession_saver_attempt,possession_saver_catch,conversion_attempt,conversion_catch,redzone_attempt,redzone_catch,player_game_count,targets,receptions,contested_receptions,contested_targets,drops,fumbles,inline_snaps,pass_blocks,pass_plays,routes,slot_snaps,wide_snaps,route_rate,slot_rate,wide_rate,contested_catch_rate,cross_success_rate,curl_success_rate,corner_success_rate,out_success_rate,post_success_rate,underneath_screen_success_rate,flat_success_rate,slant_success_rate,wr_screen_success_rate,comeback_success_rate,go_success_rate,in_success_rate,deep_success_rate,play_action_success_rate,rpo_success_rate,hurry_up_success_rate,deep_sideline_success_rate,possession_saver_success_rate,difficult_success_rate,cross_pct,corner_pct,out_pct,curl_pct,post_pct,underneath_screen_pct,flat_pct,slant_pct,wr_screen_pct,comeback_pct,go_pct,in_pct,deep_pct,play_action_pct,rpo_pct,hurry_up_pct,deep_sideline_pct,possession_saver_pct,difficult_pct,conversion_pct,redzone_pct,adot,avg_yac,avg_yacon,catch_rate,yprr,height_in,weight_lbs,40,bench,vertical,broad_jump,shuttle,3_cone
50,Kalif Raymond,WR,2022,64.00,616,638,218,34,2,5,0,12,0,47,47,27,17,0,3,1,12,10,3,0,8,5,7,5,2,2,4,4,6,4,8,8,5,4,4,2,2,2,13,7,9,12,9,2,2,2,0,3,3,6,2,0,27,15,10,9,2,0,17,62,47,3,7,0,1,2,2,346,322,159,181,0.93,0.46,0.52,0.43,0.33,0.83,0.00,0.62,0.71,1.00,1.00,0.67,1.00,0.80,0.50,1.00,0.54,0.75,1.00,1.00,0.33,0.56,0.00,0.05,0.05,0.12,0.19,0.11,0.03,0.06,0.09,0.12,0.08,0.06,0.03,0.20,0.19,0.03,0.05,0.09,0.42,0.19,0.16,0.03,9.97,4.64,0.72,0.73,1.91,68.00,180.00,NaN,NaN,NaN,NaN,NaN,NaN
66,Rashid Shaheed,WR,2022,34.00,488,401,176,31,0,2,2,4,0,17,17,7,5,0,2,2,2,2,2,2,7,5,2,2,0,0,5,4,1,0,4,4,3,3,5,3,2,1,7,6,6,6,5,1,1,0,1,1,1,3,2,0,17,13,8,6,0,0,12,34,28,0,1,0,1,0,0,193,188,50,142,0.97,0.26,0.74,0.00,1.00,1.00,1.00,0.71,1.00,0.00,0.80,0.00,1.00,1.00,0.60,0.50,0.86,0.83,1.00,1.00,0.67,0.76,0.00,0.06,0.06,0.21,0.06,0.06,0.00,0.15,0.03,0.12,0.09,0.15,0.06,0.21,0.18,0.03,0.03,0.09,0.50,0.12,0.24,0.00,11.79,6.29,1.11,0.82,2.60,70.63,185.00,NaN,10.00,NaN,NaN,NaN,NaN
84,Nick Westbrook-Ikhine,WR,2022,50.00,397,641,112,37,1,15,3,10,0,17,17,17,8,0,0,0,16,9,5,3,5,2,1,0,0,0,2,2,7,2,0,0,0,0,11,5,3,2,10,3,3,8,5,2,2,1,1,5,1,5,1,0,30,18,15,8,4,2,17,45,25,6,20,3,0,0,1,443,419,245,197,0.95,0.55,0.44,0.30,0.00,0.56,0.60,0.40,0.00,0.00,1.00,0.29,0.00,0.00,0.45,0.67,0.30,0.62,1.00,0.20,0.20,0.60,0.00,0.00,0.10,0.10,0.32,0.02,0.00,0.04,0.14,0.00,0.00,0.22,0.06,0.20,0.16,0.04,0.10,0.10,0.60,0.20,0.30,0.08,12.82,4.48,1.48,0.50,0.95,74.63,211.00,NaN,NaN,NaN,NaN,NaN,NaN
96,Kendall Hinton,WR,2022,33.00,311,303,139,52,3,4,0,5,0,13,13,12,9,1,2,0,3,2,2,1,7,6,1,0,0,0,7,6,3,3,0,0,3,3,2,1,3,2,4,2,6,6,5,2,1,3,2,1,1,2,1,0,13,7,5,4,1,0,12,31,24,0,0,3,0,0,0,314,286,164,150,0.91,0.52,0.48,0.00,0.00,0.67,0.50,0.86,0.00,0.00,0.86,1.00,0.00,1.00,0.50,0.67,0.50,0.83,0.50,1.00,0.50,0.54,0.00,0.06,0.06,0.21,0.09,0.03,0.00,0.21,0.09,0.00,0.09,0.06,0.09,0.12,0.18,0.06,0.03,0.06,0.39,0.15,0.15,0.03,9.18,5.79,2.17,0.73,1.09,70.38,193.00,NaN,NaN,NaN,NaN,NaN,NaN
146,River Cracraft,WR,2022,14.00,102,122,26,1,0,1,2,4,0,3,3,5,1,0,1,0,2,1,1,0,1,1,1,1,0,0,0,0,4,4,0,0,2,0,1,1,1,1,0,0,0,4,3,0,0,0,0,1,1,0,0,0,8,4,3,1,3,2,11,14,9,1,3,0,0,0,0,85,83,28,56,0.98,0.33,0

In [77]:
weight_ranges = [(140, 150, 4.44), (150, 160, 4.5), (160, 170, 4.53), (170, 180, 4.55),
                 (180, 190, 4.54), (190, 200, 4.54), (200, 210, 4.54), (210, 220, 4.55),
                 (220, 230, 4.55), (230, 240, 4.59), (240, 250, 4.65), (250, 260, 4.7)]

for start_weight, end_weight, median_40 in weight_ranges:
    aggregate.loc[(aggregate['weight_lbs'] > start_weight) & (aggregate['weight_lbs'] <= end_weight) & (aggregate['40'].isna()), '40'] = median_40

#### Bench press

In [78]:
weight_ranges = [(140, 150, 3), (150, 160, 5), (160, 170, 9), (170, 180, 11),
                 (180, 190, 12), (190, 200, 13), (200, 210, 14), (210, 220, 16),
                 (220, 230, 17), (230, 240, 17), (240, 250, 20), (250, 260, 23)]

for start_weight, end_weight, mean_bench in weight_ranges:
    aggregate.loc[(aggregate['weight_lbs'] > start_weight) & (aggregate['weight_lbs'] <= end_weight) & (aggregate['bench'].isna()), 'bench'] = mean_bench

#### Vertical

In [79]:
weight_ranges = [(140, 150, 32.5), (150, 160, 33), (160, 170, 34), (170, 180, 34),
                 (180, 190, 34.5), (190, 200, 35), (200, 210, 35), (210, 220, 35.5),
                 (220, 230, 35), (230, 240, 34.5), (240, 250, 34.5), (250, 260, 29)]

for start_weight, end_weight, mean_vertical in weight_ranges:
    aggregate.loc[(aggregate['weight_lbs'] > start_weight) & (aggregate['weight_lbs'] <= end_weight) & (aggregate['vertical'].isna()), 'vertical'] = mean_vertical

#### Broad jump

In [80]:
weight_ranges = [(140, 150, 114), (150, 160, 116), (160, 170, 118), (170, 180, 118),
                 (180, 190, 119), (190, 200, 120), (200, 210, 121), (210, 220, 121),
                 (220, 230, 121), (230, 240, 120), (240, 250, 126), (250, 260, 118)]

for start_weight, end_weight, mean_broad in weight_ranges:
    aggregate.loc[(aggregate['weight_lbs'] > start_weight) & (aggregate['weight_lbs'] <= end_weight) & (aggregate['broad_jump'].isna()), 'broad_jump'] = mean_broad

#### Shuttle

In [81]:
weight_ranges = [(140, 150, 4.35), (150, 160, 4.2), (160, 170, 4.22), (170, 180, 4.22),
                 (180, 190, 4.23), (190, 200, 4.24), (200, 210, 4.27), (210, 220, 4.28),
                 (220, 230, 4.26), (230, 240, 4.37), (240, 250, 4.17), (250, 260, 4.14)]

for start_weight, end_weight, mean_shuttle in weight_ranges:
    aggregate.loc[(aggregate['weight_lbs'] > start_weight) & (aggregate['weight_lbs'] <= end_weight) & (aggregate['shuttle'].isna()), 'shuttle'] = mean_shuttle

#### 3-cone

In [82]:
weight_ranges = [(140, 150, 6.88), (150, 160, 6.88), (160, 170, 6.95), (170, 180, 6.99),
                 (180, 190, 6.99), (190, 200, 7), (200, 210, 7.02), (210, 220, 7.05),
                 (220, 230, 7.04), (230, 240, 7.08), (240, 250, 7.46), (250, 260, 7.6)]

for start_weight, end_weight, mean_3_cone in weight_ranges:
    aggregate.loc[(aggregate['weight_lbs'] > start_weight) & (aggregate['weight_lbs'] <= end_weight) & (aggregate['3_cone'].isna()), '3_cone'] = mean_3_cone

In [83]:
# Find nulls
columns = aggregate.columns
field = [var for var in columns if aggregate[var].isnull().sum() != 0]
print(aggregate[field].isnull().sum())

Series([], dtype: float64)


## Column re-ordering
- Excluding 'inline_snaps', 'broken_tackles', 'drops', 'fumbles', 'dpi_drawn'
- The distributions for these fields are not varied enough

In [84]:
aggregate = aggregate[['player_name', 'player_position', 'season_year', 'player_game_count', 'receptions', 'target', 'yards', 'att_yards', 'yards_after_catch', 'yards_after_contact', 'touchdown', 
                       'routes', 'pass_plays', 'contested_receptions', 'contested_targets', 'weather_attempt', 'difficult_attempt', 'difficult_catch', 'difficult_success_rate', 'difficult_pct',
                       'weather_catch', 'qb_bf_attempt', 'qb_bf_catch', 'hurry_up_attempt', 'hurry_up_catch', 'possession_saver_attempt', 'possession_saver_catch', 'conversion_attempt', 'conversion_catch', 'redzone_attempt', 'redzone_catch', 'clutch_catch',   
                       'deep_attempt', 'deep_catch', 'deep_sideline_attempt', 'deep_sideline_catch', 'large_yac_catch', 'tackle_breaker_catch', 'beast_catch', 'play_action_attempt', 'play_action_catch', 'rpo_attempt',
                       'rpo_catch', 'cross_attempt', 'cross_catch', 'corner_attempt', 'corner_catch', 'out_attempt', 'out_catch', 'curl_attempt', 'curl_catch', 'post_attempt', 'post_catch', 'underneath_screen_attempt', 'underneath_screen_catch',  
                       'flat_attempt', 'flat_catch', 'slant_attempt', 'slant_catch', 'wr_screen_attempt', 'wr_screen_catch', 'comeback_attempt', 'comeback_catch', 'go_attempt', 'go_catch', 'in_attempt', 
                       'in_catch', 'slot_snaps', 'wide_snaps', 'cross_success_rate', 'corner_success_rate', 'out_success_rate', 'curl_success_rate', 'post_success_rate', 'underneath_screen_success_rate', 'flat_success_rate', 'slant_success_rate', 'wr_screen_success_rate'
                       , 'comeback_success_rate', 'go_success_rate', 'in_success_rate', 'deep_success_rate', 'play_action_success_rate', 'rpo_success_rate', 'hurry_up_success_rate', 'deep_sideline_success_rate'
                       , 'possession_saver_success_rate', 'route_rate', 'slot_rate', 'wide_rate', 'contested_catch_rate', 'cross_pct', 'corner_pct', 'out_pct', 'curl_pct', 'post_pct', 'underneath_screen_pct', 'flat_pct', 'slant_pct', 'wr_screen_pct'
                       , 'comeback_pct', 'go_pct', 'in_pct', 'deep_pct', 'play_action_pct', 'rpo_pct', 'hurry_up_pct', 'deep_sideline_pct', 'possession_saver_pct', 'conversion_pct', 'redzone_pct', 'adot', 'avg_yac', 'avg_yacon', 'catch_rate', 'yprr'
                       , 'height_in', 'weight_lbs', '40', 'bench', 'vertical', 'broad_jump', 'shuttle', '3_cone']]

In [85]:
aggregate.shape

(225, 124)

## Numeric column conversions

In [86]:
int_columns = ['weight_lbs', 'bench', 'broad_jump']

for col in int_columns:
    aggregate[col] = aggregate[col].fillna(0)
    aggregate[col] = aggregate[col].astype(int)

In [87]:
aggregate.dtypes

player_name                        object
player_position                    object
season_year                         int64
player_game_count                   int64
receptions                          int64
target                            float64
yards                               int64
att_yards                           int64
yards_after_catch                   int64
yards_after_contact                 int64
touchdown                           int64
routes                              int64
pass_plays                          int64
contested_receptions                int64
contested_targets                   int64
weather_attempt                     int64
difficult_attempt                   int64
difficult_catch                     int64
difficult_success_rate            float64
difficult_pct                     float64
weather_catch                       int64
qb_bf_attempt                       int64
qb_bf_catch                         int64
hurry_up_attempt                  

In [88]:
# Find nulls
columns = aggregate.columns
field = [var for var in columns if aggregate[var].isnull().sum() != 0]
print(aggregate[field].isnull().sum())

Series([], dtype: float64)


# EXPORT

In [89]:
aggregate.to_csv('../working_exports/aggregate.csv', index = False)